In [ ]:
##### Carga de librerías y carga del dataset 'df_model_xgb_rf.csv' y eliminación de filas con valores faltantes

# Carga de librerías
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

# Carga de datos
df_model_xgb = pd.read_csv(r"../data/final/1_panel/3_modelling/final_dataset_trasversal.csv")
df_model_xgb = df_model_xgb.dropna()
df_model_xgb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24232 entries, 0 to 25168
Data columns (total 72 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   dpto                     24232 non-null  int64  
 1   dominio                  24232 non-null  int64  
 2   tuvotrabajo              24232 non-null  int64  
 3   sector_trabajador        24232 non-null  int64  
 4   categoria_trabajador     24232 non-null  int64  
 5   trabajopara              24232 non-null  int64  
 6   registropersonajuridica  24232 non-null  float64
 7   cuentassunat             24232 non-null  float64
 8   pagosueldo               24232 non-null  int64  
 9   pagosalario              24232 non-null  float64
 10  pagocomision             24232 non-null  float64
 11  pagodestajo              24232 non-null  float64
 12  pagosubvencion           24232 non-null  float64
 13  pagohonorarios           24232 non-null  float64
 14  pagoganancianegocio      24

In [11]:
df_model_xgb

,dpto,dominio,tuvotrabajo,sector_trabajador,categoria_trabajador,trabajopara,registropersonajuridica,cuentassunat,pagosueldo,pagosalario,...,ingtrabw,alquiler,tiempotrabajo,horastotales_sempasada,ratiodep,region,area,afp,onp,ciiu_6c
0,1,4,1,5,3,2,3.0,99.0,1,0.0,...,13577.0,400,9.583333,40,0.400000,2,1,1,0,6
1,1,4,1,5,3,2,3.0,99.0,1,0.0,...,18221.0,150,1.333333,45,0.800000,2,1,1,0,6
2,1,4,1,6,2,99,1.0,1.0,0,0.0,...,36028.0,500,29.666666,52,0.500000,2,1,0,0,4
3,1,4,1,4,1,99,2.0,1.0,0,0.0,...,26510.0,0,41.750000,48,0.500000,2,1,0,0,4
4,1,4,1,5,3,6,1.0,1.0,1,0.0,...,12113.0,100,0.500000,44,1.000000,2,1,1,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25164,23,3,1,6,2,99,2.0,2.0,0,0.0,...,24033.0,1000,17.000000,60,0.571429,1,1,0,0,4
25165,23,3,1,8,4,6,1.0,1.0,0,1.0,...,73477.0,1000,30.000000,48,0.666667,1,1,1,0,1
25166,23,3,1,3,3,2,3.0,99.0,1,0.0,...,42659.0,1000,2.000000,42,0.666667,1,1,1,0,6
25167,9,5,1,9,4,5,1.0,1.0,0,1.0,...,19657.0,18,0.250000,52,0.333333,2,0,1,0,1


### Corremos un xgboost

In [12]:
# XGBoost_sin_rebalanceo_model

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, f1_score
import numpy as np

# 1. Dividir dataset
X = df_model_xgb.drop(columns=['status_inf'])
y = df_model_xgb['status_inf']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 2. Entrenar modelo base sin rebalanceo
XGBoost_sin_rebalanceo_model = XGBClassifier(
    eval_metric='logloss',
    random_state=42,
)

XGBoost_sin_rebalanceo_model.fit(X_train, y_train)

# 3. Predicciones de probabilidad
y_proba = XGBoost_sin_rebalanceo_model.predict_proba(X_test)[:, 1]

# 4. Aplicar umbral fijo = 0.18
y_pred_adjusted = (y_proba > 0.18).astype(int)

# 5. Métricas y reporte
print("=== Reporte de Clasificación (umbral fijo = 0.18) ===")
print(classification_report(y_test, y_pred_adjusted, target_names=["Clase 0", "Clase 1"], digits=4))

accuracy_xgb = accuracy_score(y_test, y_pred_adjusted)
auc_roc_xgb = roc_auc_score(y_test, y_pred_adjusted)
f1_xgb = f1_score(y_test, y_pred_adjusted)

print(f"\nXGBoost (sin rebalanceo, umbral 0.18) - Accuracy: {accuracy_xgb:.3f}, AUC-ROC: {auc_roc_xgb:.3f}, F1: {f1_xgb:.3f}")


=== Reporte de Clasificación (umbral fijo = 0.18) ===
              precision    recall  f1-score   support

     Clase 0     0.8966    0.7344    0.8074      3991
     Clase 1     0.3283    0.6051    0.4256       856

    accuracy                         0.7116      4847
   macro avg     0.6124    0.6698    0.6165      4847
weighted avg     0.7962    0.7116    0.7400      4847


XGBoost (sin rebalanceo, umbral 0.18) - Accuracy: 0.712, AUC-ROC: 0.670, F1: 0.426


In [13]:
# XGBoost_con_scale_pos_weight_model, validación cruzada (4 folds) + logloss + umbral 0.18
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_predict
from sklearn.metrics import (
    classification_report, accuracy_score, roc_auc_score, f1_score, recall_score
)
import pandas as pd
import numpy as np

X = df_model_xgb.drop(columns=['status_inf'])
y = df_model_xgb['status_inf']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f"scale_pos_weight = {scale_pos_weight:.2f}")

XGBoost_con_scale_pos_weight_model = XGBClassifier(
    eval_metric='logloss',
    random_state=42,
    scale_pos_weight=scale_pos_weight
)

kf = KFold(n_splits=4, shuffle=True, random_state=42)
y_proba_cv = cross_val_predict(
    XGBoost_con_scale_pos_weight_model, X_train, y_train, cv=kf, method='predict_proba'
)[:, 1]
y_pred_cv = (y_proba_cv >= 0.18).astype(int)

accuracy_cv = accuracy_score(y_train, y_pred_cv)
auc_roc_cv = roc_auc_score(y_train, y_proba_cv)
f1_cv = f1_score(y_train, y_pred_cv)
recall_cv_1 = recall_score(y_train, y_pred_cv, pos_label=1)

print("\n=== Resultados de Cross-Validation (4 folds) ===")
print(f"Accuracy promedio: {accuracy_cv:.3f}")
print(f"AUC-ROC promedio: {auc_roc_cv:.3f}")
print(f"F1 promedio: {f1_cv:.3f}")
print(f"Recall (Clase 1) promedio: {recall_cv_1:.3f}")

XGBoost_con_scale_pos_weight_model.fit(X_train, y_train)

y_proba_train = XGBoost_con_scale_pos_weight_model.predict_proba(X_train)[:, 1]
y_pred_train = (y_proba_train >= 0.18).astype(int)

y_proba_test = XGBoost_con_scale_pos_weight_model.predict_proba(X_test)[:, 1]
y_pred_test = (y_proba_test >= 0.18).astype(int)

print("\n=== Reporte en Test ===")
print(classification_report(y_test, y_pred_test, target_names=["Clase 0", "Clase 1"], digits=4))

accuracy_xgb = accuracy_score(y_test, y_pred_test)
auc_roc_xgb = roc_auc_score(y_test, y_proba_test)
f1_xgb = f1_score(y_test, y_pred_test)

print(f"XGBoost (Test) - Accuracy: {accuracy_xgb:.3f}, AUC-ROC: {auc_roc_xgb:.3f}, F1: {f1_xgb:.3f}")

recall_train_0 = recall_score(y_train, y_pred_train, pos_label=0)
recall_train_1 = recall_score(y_train, y_pred_train, pos_label=1)
recall_test_0 = recall_score(y_test, y_pred_test, pos_label=0)
recall_test_1 = recall_score(y_test, y_pred_test, pos_label=1)

recall_table = pd.DataFrame({
    'Clase 0': [recall_train_0, recall_test_0],
    'Clase 1': [recall_train_1, recall_test_1]
}, index=['Train', 'Test'])

print("\n=== Tabla de Recall por clase ===")
print(recall_table.round(4))


scale_pos_weight = 4.66

=== Resultados de Cross-Validation (4 folds) ===
Accuracy promedio: 0.563
AUC-ROC promedio: 0.728
F1 promedio: 0.393
Recall (Clase 1) promedio: 0.801

=== Reporte en Test ===
              precision    recall  f1-score   support

     Clase 0     0.9330    0.4640    0.6198      3991
     Clase 1     0.2526    0.8446    0.3889       856

    accuracy                         0.5313      4847
   macro avg     0.5928    0.6543    0.5044      4847
weighted avg     0.8128    0.5313    0.5790      4847

XGBoost (Test) - Accuracy: 0.531, AUC-ROC: 0.731, F1: 0.389

=== Tabla de Recall por clase ===
       Clase 0  Clase 1
Train   0.5367   0.9994
Test    0.4640   0.8446


In [14]:
# XGBoost_con_SMOTE_model, validación cruzada (4 folds) + logloss + umbral 0.18
# ==============================

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_predict
from sklearn.metrics import (
    classification_report, accuracy_score, roc_auc_score, f1_score, recall_score
)
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import pandas as pd
import numpy as np

X = df_model_xgb.drop(columns=['status_inf'])
y = df_model_xgb['status_inf']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Distribución original en train: {np.bincount(y_train)}")

smote = SMOTE(random_state=42, sampling_strategy='auto')
XGBoost_con_SMOTE_model = XGBClassifier(
    eval_metric='logloss',
    random_state=42,
    scale_pos_weight=None
)

pipeline = Pipeline(steps=[('smote', smote), ('xgb', XGBoost_con_SMOTE_model)])

kf = KFold(n_splits=4, shuffle=True, random_state=42)

y_proba_cv = cross_val_predict(pipeline, X_train, y_train, cv=kf, method='predict_proba')[:, 1]
y_pred_cv = (y_proba_cv >= 0.18).astype(int)

accuracy_cv = accuracy_score(y_train, y_pred_cv)
auc_roc_cv = roc_auc_score(y_train, y_proba_cv)
f1_cv = f1_score(y_train, y_pred_cv)
recall_cv_1 = recall_score(y_train, y_pred_cv, pos_label=1)

print("\n=== Resultados Cross-Validation (4 folds, con SMOTE) ===")
print(f"Accuracy promedio: {accuracy_cv:.3f}")
print(f"AUC-ROC promedio: {auc_roc_cv:.3f}")
print(f"F1 promedio: {f1_cv:.3f}")
print(f"Recall (Clase 1) promedio: {recall_cv_1:.3f}")

X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)
print(f"Después de SMOTE: {np.bincount(y_train_sm)}")

XGBoost_con_SMOTE_model.fit(X_train_sm, y_train_sm)

y_proba_train = XGBoost_con_SMOTE_model.predict_proba(X_train)[:, 1]
y_pred_train = (y_proba_train >= 0.18).astype(int)

y_proba_test = XGBoost_con_SMOTE_model.predict_proba(X_test)[:, 1]
y_pred_test = (y_proba_test >= 0.18).astype(int)

print("\n=== Reporte en Test ===")
print(classification_report(y_test, y_pred_test, target_names=["Clase 0", "Clase 1"], digits=4))

accuracy_xgb = accuracy_score(y_test, y_pred_test)
auc_roc_xgb = roc_auc_score(y_test, y_proba_test)
f1_xgb = f1_score(y_test, y_pred_test)

print(f"XGBoost + SMOTE (Test) - Accuracy: {accuracy_xgb:.3f}, AUC-ROC: {auc_roc_xgb:.3f}, F1: {f1_xgb:.3f}")

recall_train_0 = recall_score(y_train, y_pred_train, pos_label=0)
recall_train_1 = recall_score(y_train, y_pred_train, pos_label=1)
recall_test_0 = recall_score(y_test, y_pred_test, pos_label=0)
recall_test_1 = recall_score(y_test, y_pred_test, pos_label=1)

recall_table = pd.DataFrame({
    'Clase 0': [recall_train_0, recall_test_0],
    'Clase 1': [recall_train_1, recall_test_1]
}, index=['Train', 'Test'])

print("\n=== Tabla de Recall por clase ===")
print(recall_table.round(4))



Distribución original en train: [15962  3423]

=== Resultados Cross-Validation (4 folds, con SMOTE) ===
Accuracy promedio: 0.709
AUC-ROC promedio: 0.736
F1 promedio: 0.424
Recall (Clase 1) promedio: 0.608
Después de SMOTE: [15962 15962]

=== Reporte en Test ===
              precision    recall  f1-score   support

     Clase 0     0.8988    0.7254    0.8028      3991
     Clase 1     0.3260    0.6192    0.4271       856

    accuracy                         0.7066      4847
   macro avg     0.6124    0.6723    0.6150      4847
weighted avg     0.7976    0.7066    0.7365      4847

XGBoost + SMOTE (Test) - Accuracy: 0.707, AUC-ROC: 0.739, F1: 0.427

=== Tabla de Recall por clase ===
       Clase 0  Clase 1
Train   0.8213   0.9454
Test    0.7254   0.6192
